In [1]:
from utils.alloys import alloy_to_1d_tensor, get_elements_and_compositions, element_to_index
from utils.constants import alloy_max_len, n_elements
from utils.ml import *
from utils.dataframes import dfs_tabs
import matplotlib.pyplot as plt

2023-09-17 15:45:46.018985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-17 15:45:46.562047: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-17 15:45:47.672972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.0/include:/usr/local/cuda-11.0/lib64:
2023-09-17 15:45:47.673409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerro

In [2]:
df_train = pd.read_csv("train_final_df.csv")
df_test = pd.read_csv("test_final_df.csv")

# Print the shape of the training and testing sets to check the split sizes
print("Training set shape:", df_train.shape)
print("Testing set shape:", df_test.shape)

Training set shape: (1269, 3)
Testing set shape: (318, 3)


In [3]:
# constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ideally a max factor is from the training dataset
max_factor = max(df_train["actual_d_max"])
print(max_factor)

35.0


In [4]:
# creating the input and output data for the model
X_train = []
y_train = []

X_test = []
y_test = []

elements_vector_df = pd.read_csv("utils/elements_wvmodel_2016_12-15-20.csv")

def alloy_to_vectorized_tensor(alloy_str, alloy_max_len = alloy_max_len):
    tensor = torch.zeros(alloy_max_len, 200)
    elements, compositions = get_elements_and_compositions(alloy_str)
    i = 0
    j = 0
    for idx in range(0, len(elements) + len(compositions), 2):
        tensor[j, :] = torch.tensor(np.array(elements_vector_df[elements[i]]))
        tensor[j, int(compositions[i])] = 1
        i += 1
        j += 2
    return tensor


for i in range(df_train.shape[0]):
    X_train.append(alloy_to_vectorized_tensor(df_train.loc[i, "bmg_alloy"]))
    y_train.append(df_train.loc[i, "actual_d_max"] / max_factor)
    
X_train = torch.stack(X_train)
y_train = torch.tensor(y_train, dtype=torch.float32)

print(X_train.shape, X_train.dtype, y_train.shape, min(y_train).item(), max(y_train).item())

for i in range(df_test.shape[0]):
    X_test.append(alloy_to_vectorized_tensor(df_test.loc[i, "bmg_alloy"]))
    y_test.append(df_test.loc[i, "actual_d_max"] / max_factor)
    
X_test = torch.stack(X_test)
y_test = torch.tensor(y_test, dtype=torch.float32)

print(X_test.shape, min(y_test).item(), max(y_test).item())

torch.Size([1269, 20, 200]) torch.float32 torch.Size([1269]) 0.0 1.0
torch.Size([318, 20, 200]) 0.0 0.8571428656578064


In [8]:
import torch

# Assuming X_train and X_test are Python lists of data
# Convert them to PyTorch tensors
X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)

# Calculate mean and standard deviation from the training data along axis 0
mean = torch.mean(X_train, dim=0)
std = torch.std(X_train, dim=0)

epsilon = 1e-8  # You can adjust this value if needed
std = std + epsilon
mean = mean + epsilon
print(mean, std)

# Normalize the data
normalized_X_train = (X_train - mean) / std
normalized_X_test = (X_test - mean) / std

tensor([[ 8.0976e-08,  2.1794e-08,  2.0667e-09,  ..., -1.3152e-08,
          9.9200e-08,  1.5030e-09],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan],
        [ 8.5673e-08, -7.5152e-10, -2.6867e-08,  ...,  8.9806e-08,
         -8.2667e-08, -2.2545e-08],
        ...,
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan,  ...,         nan,
                 nan,         nan]]) tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
        [   nan,    nan,    nan,  .

In [6]:
y_train.min(), y_train.max()

(tensor(0.), tensor(1.))

In [7]:
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Take the output of the last time step
        out = self.sigmoid(out)
        return out

# Define the model parameters
input_size = 200
hidden_size = 128
num_layers = 3
output_size = 1

# Create the model
model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
# criterion = nn.BCELoss()  # R2 Score: 0.69
# criterion = nn.MSELoss() # R2 Score: 0.0146
# criterion = nn.KLDivLoss()
criterion = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# Convert your data to PyTorch tensors
X_train = torch.tensor(normalized_X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_train = y_train.view(-1, 1)
# Training loop
num_epochs = 5000
print_every = 100
plot_loss = []
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % print_every == 0:
        print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item():.4f}")
    plot_loss.append(loss.item())

RuntimeError: all elements of input should be between 0 and 1

In [ ]:
plt.plot(range(len(plot_loss)), plot_loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")

In [ ]:
torch.save(model.state_dict(), "rnn_trainset_trained_method_3.pt")

In [ ]:
predicted_labels = []
model.eval()
with torch.no_grad():
    predicted_labels = model(X_test)

In [ ]:
r2 = r2_score(np.array(y_test)*max_factor, np.array(predicted_labels).reshape(-1)*max_factor)
print(f"R2 Score: {r2:.4f}")

In [ ]:
sns.regplot(x=np.array(y_test)*max_factor, y=np.array(predicted_labels).reshape(-1)*max_factor)

In [ ]:
all_rnn_train_output = []
for i in range(df_train.shape[0]):
    alloy_tensor = alloy_to_vectorized_tensor(df_train.loc[i, "bmg_alloy"])
    output = model(alloy_tensor.unsqueeze(0))
    all_rnn_train_output.append(output.squeeze().item() * max_factor)

all_rnn_test_output = []
for i in range(df_test.shape[0]):
    alloy_tensor =  alloy_to_vectorized_tensor(df_test.loc[i, "bmg_alloy"])
    output = model(alloy_tensor.unsqueeze(0))
    all_rnn_test_output.append(output.squeeze().item() * max_factor)
    
new_train = df_train
new_test = df_test


new_train["rnn_encoding"] = all_rnn_train_output
new_test["rnn_encoding"] = all_rnn_test_output
    
dfs = [new_train, new_test]
sheets = ["train", "test"]
dfs_tabs(dfs, sheets, 'dataset_rnn.xlsx')

In [ ]:
df_train = pd.read_excel("dataset_rnn.xlsx", sheet_name="train")
df_test = pd.read_excel("dataset_rnn.xlsx", sheet_name="test")
display(df_train)
display(df_test)

In [ ]:
df_train = pd.read_excel("dataset_rnn.xlsx", index_col=0, sheet_name="train")
df_test = pd.read_excel("dataset_rnn.xlsx", index_col=0, sheet_name="test")
# df_test = df_test.drop(["predicted_d_max"], axis=1)
df_train = df_train.drop(["paper_sno"], axis=1)
df_test = df_test.drop(["paper_sno"], axis=1)
X_train, y_train = df_train.loc[:, df_train.columns != "actual_d_max"], pd.DataFrame(df_train["actual_d_max"])
X_test, y_test = df_test.loc[:, df_test.columns != "actual_d_max"], pd.DataFrame(df_test["actual_d_max"])

R2 = []
ADJR2 = []
RMSE = []
names = []
TIME = []

for name, model in tqdm(REGRESSORS):
    start = time.time()
    pipe = Pipeline(steps=[
                        ("classifier", model()),
                    ]
                )
    try:
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        r_squared = r2_score(y_test, y_pred)
        print(name, r_squared)
        adj_rsquared = adjusted_rsquared(
            r_squared, X_test.shape[0], X_test.shape[1]
        )
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        names.append(name)
        R2.append(r_squared)
        ADJR2.append(adj_rsquared)
        RMSE.append(rmse)
        TIME.append(time.time() - start)
    except Exception as exception:
        print(name + " model failed to execute")
        print(exception)
    scores = {
                "Model": names,
                "Adjusted R-Squared": ADJR2,
                "R-Squared": R2,
                "RMSE": RMSE,
                "Time Taken": TIME,
            }
    scores = pd.DataFrame(scores)
    scores = scores.sort_values(by = "Adjusted R-Squared", ascending = False).set_index("Model")

In [ ]:
display(scores)

In [ ]:
df_train = pd.read_excel("dataset_rnn.xlsx", index_col=0, sheet_name="train")
df_test = pd.read_excel("dataset_rnn.xlsx", index_col=0, sheet_name="test")
# df_test = df_test.drop(["predicted_d_max"], axis=1)
df_train = df_train.drop(["paper_sno"], axis=1)
df_test = df_test.drop(["paper_sno"], axis=1)
X_train, y_train = df_train.loc[:, df_train.columns != "actual_d_max"], pd.DataFrame(df_train["actual_d_max"])
X_test, y_test = df_test.loc[:, df_test.columns != "actual_d_max"], pd.DataFrame(df_test["actual_d_max"])

R2 = []
ADJR2 = []
RMSE = []
names = []
TIME = []

for transformer_method_name, transformer_method in tqdm(TRANSFOMER_METHODS):
    for name, model in tqdm(REGRESSORS):
        start = time.time()
        X_transformer = transformer_method()
        y_transformer = transformer_method()
        transformed_X_train = pd.DataFrame(X_transformer.fit_transform(X_train), columns = X_train.columns)
        transformed_X_test = pd.DataFrame(X_transformer.transform(X_test), columns = X_test.columns)

        transformed_y_train = pd.DataFrame(y_transformer.fit_transform(y_train), columns = y_train.columns)
        transformed_y_test = pd.DataFrame(y_transformer.transform(y_test), columns = y_test.columns)
        pipe = Pipeline(steps=[
                            ("classifier", model()),
                        ]
                    )
        try:
            pipe.fit(transformed_X_train, transformed_y_train)
            transformed_y_pred = pipe.predict(transformed_X_test)
            r_squared = r2_score(transformed_y_test, transformed_y_pred)
            adj_rsquared = adjusted_rsquared(
                r_squared, transformed_X_test.shape[0], transformed_X_test.shape[1]
            )
            rmse = np.sqrt(mean_squared_error(transformed_y_test, transformed_y_pred))
            names.append(name + " (" + transformer_method_name + ")")
            print(name + " (" + transformer_method_name + ")", r_squared)
            R2.append(r_squared)
            ADJR2.append(adj_rsquared)
            RMSE.append(rmse)
            TIME.append(time.time() - start)
        except Exception as exception:
            print(name + " (" + transformer_method_name + ")" + " model failed to execute")
            print(exception)
        scores = {
                    "Model": names,
                    "Adjusted R-Squared": ADJR2,
                    "R-Squared": R2,
                    "RMSE": RMSE,
                    "Time Taken": TIME,
                }
        scores = pd.DataFrame(scores)
        scores = scores.sort_values(by = "Adjusted R-Squared", ascending = False).set_index("Model")

In [ ]:
display(scores[:30])